In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv

def readGz(path):
    for l in gzip.open(path, 'rt',encoding='utf8'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [2]:
import random
import scipy
import tensorflow as tf
from collections import defaultdict

userIDs = {}
recipeIDs = {}
interactions = []

for user,recipe,d in readCSV("C:/Users/brand/Desktop/trainInteractions.csv.gz"):
    u = user
    i = recipe
    r = int(d['rating'])
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in recipeIDs: recipeIDs[i] = len(recipeIDs)
    interactions.append((u,i,r))



In [17]:
import pandas as pd 

interTrain = interactions[0:490000]
interVal = interactions[490000:]

class LatentFactorModelBiasOnly(tf.keras.Model):
    def __init__(self, mu, lamb):
        super(LatentFactorModelBiasOnly, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(recipeIDs)],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i]
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        pred = self.alpha + beta_u + beta_i
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)
    

In [4]:
recipesPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interTrain:
    recipesPerUser[u].append(i)
    usersPerItem[i].append(u)

In [7]:
mu = sum([r for _,_,r in interactions]) / len(interactions)

optimizer = tf.keras.optimizers.Adam(0.01)

In [23]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(modelLFM.betaU)
        self.betaI = tf.Variable(modelLFM.betaI)
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(recipeIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [18]:
modelLFM = LatentFactorModelBiasOnly(mu, 0.00001)

In [12]:
def trainingStep(model, interactions):
    Nsamples = 40000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(recipeIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [19]:
for i in range(100):
    obj = trainingStep(modelLFM, interTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.43044338
iteration 20, objective = 0.42071941
iteration 30, objective = 0.41952595
iteration 40, objective = 0.3871444
iteration 50, objective = 0.3921595
iteration 60, objective = 0.38936442
iteration 70, objective = 0.38608065
iteration 80, objective = 0.37644282
iteration 90, objective = 0.3802232
iteration 100, objective = 0.37012932


In [20]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

labels = [r for _,_,r in interVal]
print(modelLFM.betaU)
#preds = [modelLFM.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

<tf.Variable 'Variable:0' shape=(13533,) dtype=float32, numpy=
array([ 4.5745865e-02, -7.4492194e-02,  7.2085306e-02, ...,
        5.7386546e-07, -3.3690390e-07, -1.1596380e-06], dtype=float32)>


In [14]:
print("MSE")
print(MSE(preds,labels))

MSE
0.8586860257787636


In [11]:
def hyper_parameter_search(lambs,factors):
    lamb_best = 0
    factor_best = 0
    msebest = 1000000
    mselst = []
    
    for i in lambs:
        for j in factors:
            
            modelSearch = LatentFactorModel(mu,j,i)
            optimizer = tf.keras.optimizers.Adam(0.1)
            
            for k in range(100):
                obj = trainingStep(modelSearch, interTrain)
                if (k % 10 == 9): print("iteration " + str(1+k) + ", objective = " + str(obj))
                    
            predictions = [modelSearch.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]
            mse = MSE(predictions,labels)
            
            if mse < msebest:
                msebest = mse
                lamb_best = i
                factor_best = j
                mselst.append(mse)
            
            print('epoch complete')
            print(msebest)

ldas = [0.001,.0001,.00001,.000001,.0000001]
facts = [1,5,10,20]

hyper_parameter_search(ldas,facts)

iteration 10, objective = 0.64266586
iteration 20, objective = 0.4939641
iteration 30, objective = 0.46893013
iteration 40, objective = 0.4563098
iteration 50, objective = 0.45187622
iteration 60, objective = 0.45904458
iteration 70, objective = 0.47083402
iteration 80, objective = 0.44048795
iteration 90, objective = 0.45111492
iteration 100, objective = 0.45791262
epoch complete
0.8843123215877902
iteration 10, objective = 0.69566244
iteration 20, objective = 0.7033106
iteration 30, objective = 0.6053941
iteration 40, objective = 0.49411774
iteration 50, objective = 0.46758276
iteration 60, objective = 0.4660319
iteration 70, objective = 0.45992422
iteration 80, objective = 0.45778802
iteration 90, objective = 0.4696449
iteration 100, objective = 0.47381392
epoch complete
0.8843123215877902
iteration 10, objective = 1.1700332
iteration 20, objective = 1.4851098
iteration 30, objective = 0.8698727
iteration 40, objective = 0.6126227
iteration 50, objective = 0.51892066
iteration 60, o

KeyboardInterrupt: 

In [15]:
def early_stopping(lamb,factors):
    
    mselst = []
    
    modelEarly = LatentFactorModel(mu,factors,lamb)
    optimizer = tf.keras.optimizers.Adam(0.1)
    
    for k in range(150):
        obj = trainingStep(modelEarly, interTrain)
        if (k % 10 == 9): 
            print("iteration " + str(1+k) + ", objective = " + str(obj))
            predictions = [modelEarly.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]
            mse = MSE(predictions,labels)
            
            mselst.append(mse)
                
    return mselst
    
es = early_stopping(0.0001,5)

iteration 10, objective = 0.9533016
iteration 20, objective = 0.70705557
iteration 30, objective = 0.57096374
iteration 40, objective = 0.5224955
iteration 50, objective = 0.49905178
iteration 60, objective = 0.47374216
iteration 70, objective = 0.484395
iteration 80, objective = 0.47055963
iteration 90, objective = 0.4751298
iteration 100, objective = 0.462395
iteration 110, objective = 0.47531345
iteration 120, objective = 0.45684645
iteration 130, objective = 0.44644043
iteration 140, objective = 0.46297738
iteration 150, objective = 0.46665737


In [16]:
print(es)

[0.9150879531896458, 0.880563176341879, 0.8718634677343702, 0.8649535660078818, 0.859584288954288, 0.8591463705900255, 0.8607234897648958, 0.8588737227050982, 0.8580928680701345, 0.8574674211289476, 0.8569189341243857, 0.8567934854262762, 0.8540513295165436, 0.8553620975047661, 0.8552463156999844]


In [26]:
modelLarge = LatentFactorModel(mu, 10, 0.00018)
optimizer = tf.keras.optimizers.Adam(0.01)

for i in range(1000):
    obj = trainingStep(modelLarge, interTrain)
    if (i % 10 == 9): 
        print("iteration " + str(i+1) + ", objective = " + str(obj))
    
    if i == 100:
        optimizer = tf.keras.optimizers.Adam(0.001)
        predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

        ms = MSE(predictions,labels)
        print(ms)
        
    elif i == 250:
        optimizer = tf.keras.optimizers.Adam(0.001)
        optimizer = tf.keras.optimizers.Adam(0.001)
        predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

        ms = MSE(predictions,labels)
        print(ms)
        
    elif i == 350:
        optimizer = tf.keras.optimizers.Adam(0.0001)
        predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

        ms = MSE(predictions,labels)
        print(ms)
        
    
    elif i == 550:
        optimizer = tf.keras.optimizers.Adam(0.0001)
        predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

        ms = MSE(predictions,labels)
        print(ms)
        
    elif i == 750:
        optimizer = tf.keras.optimizers.Adam(0.0001)
        predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

        ms = MSE(predictions,labels)
        print(ms)
        
    elif i == 550:
        optimizer = tf.keras.optimizers.Adam(0.001)        
        predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

        ms = MSE(predictions,labels)
        print(ms)


iteration 10, objective = 0.4054548
iteration 20, objective = 0.403812
iteration 30, objective = 0.39680743
iteration 40, objective = 0.39499995
iteration 50, objective = 0.40249184
iteration 60, objective = 0.39527678
iteration 70, objective = 0.39700308
iteration 80, objective = 0.37521222
iteration 90, objective = 0.3972501
iteration 100, objective = 0.3959826
0.7941164413686111
iteration 110, objective = 0.3879762
iteration 120, objective = 0.39252177
iteration 130, objective = 0.40092695
iteration 140, objective = 0.38103923
iteration 150, objective = 0.38239542
iteration 160, objective = 0.38928697
iteration 170, objective = 0.3810575
iteration 180, objective = 0.38608503
iteration 190, objective = 0.39172852
iteration 200, objective = 0.3820506
iteration 210, objective = 0.380563
iteration 220, objective = 0.38460937
iteration 230, objective = 0.4000124
iteration 240, objective = 0.38348714
iteration 250, objective = 0.39917007
0.7930055500330001
iteration 260, objective = 0.373

In [27]:

predictions = [modelLarge.predict(userIDs[u],recipeIDs[i]).numpy() for u,i,_ in interVal]

ms = MSE(predictions,labels)
print(ms)

0.7922724971529275


In [100]:
for i in range(100):
    obj = trainingStep(modelLarge, interactions)
    if (i % 10 == 9): 
        print("iteration " + str(i+1) + ", objective = " + str(obj))
    
    if i == 1:
        optimizer = tf.keras.optimizers.Adam(0.00001)
        
    elif i == 185:
        optimizer = tf.keras.optimizers.Adam(0.0001)

iteration 10, objective = 0.3887083
iteration 20, objective = 0.38732338
iteration 30, objective = 0.3794976
iteration 40, objective = 0.3888142
iteration 50, objective = 0.38962168
iteration 60, objective = 0.3839594
iteration 70, objective = 0.38097835
iteration 80, objective = 0.38253117
iteration 90, objective = 0.38363507
iteration 100, objective = 0.3815946


In [29]:
allRatings = []

userRatings = defaultdict(list)
recipeRatings = defaultdict(list)

for user,recipe,d in readCSV("C:/Users/brand/Desktop/trainInteractions.csv.gz"):
    r = int(d['rating'])
    allRatings.append(r)
    userRatings[user].append(r)
    recipeRatings[recipe].append(r)

globalAverageuser = sum(allRatings) / len(allRatings)


userAverage = {}
itemAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
    
for i in recipeRatings:
    itemAverage[i] = sum(recipeRatings[i])/len(recipeRatings[i])


In [30]:
predictions = open("C:/Users/brand/Desktop/predictions_Rated.txt", 'w')
for l in open("C:/Users/brand/Desktop/stub_Rated.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    
    if i not in recipeIDs and u in userAverage:
        
        predictions.write(u + '-' + i + "," + str(userAverage[u]) + "\n")
        continue
        
    elif u not in userIDs and i in itemAverage:
        predictions.write(u+'-'+i+"," + str(itemAverage[i]) + "\n")
        continue
        
    elif u in userIDs and i in recipeIDs:
        predictions.write(u + '-' + i + "," + str(modelLarge.predict(userIDs[u],recipeIDs[i]).numpy()) + "\n")
        continue
    
    else:
        predictions.write(u + '-' + i + "," + str(globalAverageuser) + "\n")

predictions.close()